In [10]:
import os
import copy
import joblib
import shutil
import numpy as np
import pandas as pd
import networkx as nx
from tqdm.notebook import tqdm
from traffic.core import Flight
from traffic.core import Traffic
from matplotlib import pyplot as plt
from xgboost import XGBRegressor as XGBR

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
pd.set_option("display.max_columns", 100)
data_folder = os.path.join(os.getcwd(), "data")
submissions_folder = os.path.join(data_folder, "submissions")

In [11]:
xgb_X_y_df_cha = joblib.load("best_xgb_X_y_df_cha_v4.pkl")
xgb_X_y_df_final_sub = joblib.load("best_xgb_X_y_df_final_sub_v4.pkl")

results_df = joblib.load("results_df_v4.pkl")
display(results_df.sort_values(by="RMSE_avg").head())
learning_rate, n_estimators, max_depth, col_sample, corr_th, n_quantiles = results_df.sort_values(
    by="RMSE_avg"
).iloc[0].values[:6]

X = xgb_X_y_df_cha[xgb_X_y_df_cha.columns[1:-1]].values
y = xgb_X_y_df_cha[xgb_X_y_df_cha.columns[-1]].values

best_reg = XGBR(
    n_jobs=-1,
    max_depth=max_depth, 
    n_estimators=n_estimators,
    learning_rate=learning_rate,
    colsample_bytree=col_sample,
    colsample_bylevel=col_sample,
    colsample_bynode=col_sample,
).fit(X, y)

,learning_rate,n_estimators,max_depth,col_sample,corr_th,n_quantiles,RMSE_avg,RMSE_std,time_s,rank_1_feature,rank_1_importance,rank_2_feature,rank_2_importance,rank_3_feature,rank_3_importance,rank_4_feature,rank_4_importance,rank_5_feature,rank_5_importance,rank_6_feature,rank_6_importance,rank_7_feature,rank_7_importance,rank_8_feature,rank_8_importance,rank_9_feature,rank_9_importance,rank_10_feature,rank_10_importance,rank_11_feature,rank_11_importance,rank_12_feature,rank_12_importance,rank_13_feature,rank_13_importance,rank_14_feature,rank_14_importance,rank_15_feature,rank_15_importance,rank_16_feature,rank_16_importance,rank_17_feature,rank_17_importance,rank_18_feature,rank_18_importance,rank_19_feature,rank_19_importance,rank_20_feature,rank_20_importance
1,0.07,920,9,0.90,0.90,5,2759.864316,17.841704,60.688274,AIRCRAFT_Wingspan,0.397368,AIRCRAFT_Wheelbase_ft,0.367163,AIRCRAFT_Main_Gear_Width_ft,0.070534,AIRCRAFT_Tail_Height_at_OEW_ft,0.069549,AIRCRAFT_MAX_OPE_SPEED,0.027792,AIRCRAFT_Approach_Speed_knot,0.018389,AIRCRAFT_Ambient Baro Min (kPa),0.015450,AIRLINE_airline_5543e4dc327359ffaf5b9c0e6faaf0e1,0.010949,AIRCRAFT_Fuel Arom Min (%),0.005277,flown_distance,0.001586,total_distance,0.001542,AIRCRAFT_MAX_OPE_ALTI,0.001090,AIRCRAFT_aircraft_type_other,0.000966,FUEL_total_consumption_replace,0.000803,AIRCRAFT_NOx Dp/Foo Min (g/kN),0.000724,AIRLINE_airline_2d5def0a5a844b343ba1b7cc9cb28fa9,0.000637,AIRCRAFT_TRAIN_ON,0.000595,AIRCRAFT_Ambient Temp Min (K),0.000557,AIRCRAFT_Humidity Min (kg/kg),0.000544,AIRLINE_airline_5d407cb11cc29578cc3e292e743f5393,0.000523
2,0.03,330,13,0.80,0.80,5,2834.535553,18.991700,152.827750,AIRCRAFT_Wheelbase_ft,0.631158,AIRCRAFT_Tail_Height_at_OEW_ft,0.233029,AIRCRAFT_NOx EI C/O (g/kg),0.052206,AIRCRAFT_SN Idle,0.012153,AIRCRAFT_NOx EI App (g/kg),0.009929,AIRCRAFT_SN App,0.008311,AIRCRAFT_Pressure Ratio,0.008310,AIRCRAFT_Approach_Speed_knot,0.007525,AIRLINE_airline_5543e4dc327359ffaf5b9c0e6faaf0e1,0.007347,AIRCRAFT_Ambient Baro Min (kPa),0.006237,AIRCRAFT_Fuel Arom Min (%),0.005106,AIRCRAFT_HC Dp/Foo Characteristic (g/kN),0.001801,AIRCRAFT_aircraft_type_other,0.001686,flown_distance,0.001439,AIRCRAFT_Fuel H/C Ratio Min,0.001416,AIRCRAFT_Num_Engines,0.001301,AIRCRAFT_NOx EI Idle (g/kg),0.001186,total_distance,0.000947,AIRLINE_airline_5d407cb11cc29578cc3e292e743f5393,0.000929,AIRLINE_airline_2d5def0a5a844b343ba1b7cc9cb28fa9,0.000865
5,0.20,580,14,0.85,0.90,5,3001.496033,12.091935,1389.164687,AIRCRAFT_MAX_OPE_SPEED,0.377724,AIRCRAFT_Wheelbase_ft,0.338444,AIRCRAFT_Wingspan,0.221919,AIRLINE_airline_5543e4dc327359ffaf5b9c0e6faaf0e1,0.026177,AIRCRAFT_Approach_Speed_knot,0.005640,AIRCRAFT_Main_Gear_Width_ft,0.005640,AIRCRAFT_Ambient Baro Min (kPa),0.005476,AIRCRAFT_Tail_Height_at_OEW_ft,0.005229,AIRCRAFT_SN Idle,0.004712,total_distance,0.001299,AIRCRAFT_aircraft_type_other,0.001070,AIRLINE_airline_2d5def0a5a844b343ba1b7cc9cb28fa9,0.001065,flown_distance,0.000957,AIRCRAFT_MAX_OPE_ALTI,0.000640,track_y_quant_2_norm_stdv,0.000464,FUEL_total_consumption_replace,0.000449,AIRLINE_airline_3922524069809ac4326134429751e26f,0.000222,AIRCRAFT_TRAIN_ON,0.000214,AIRLINE_airline_5d407cb11cc29578cc3e292e743f5393,0.000188,AIRPORT_pageRank_ades,0.000177
0,0.40,810,17,0.95,0.85,5,3382.629700,28.930177,841.577657,AIRCRAFT_Wheelbase_ft,0.899433,AIRLINE_airline_5543e4dc327359ffaf5b9c0e6faaf0e1,0.054058,AIRCRAFT_Approach_Speed_knot,0.012590,AIRCRAFT_Ambient Baro Min (kPa),0.009271,AIRCRAFT_Rated Thrust (kN),0.007447,AIRCRAFT_TRAIN_ON,0.002967,flown_distance,0.002601,AIRLINE_airline_3922524069809ac4326134429751e26f,0.002284,AIRLINE_airline_5d407cb11cc29578cc3e292e743f5393,0.002101,FUEL_total_consumption_replace,0.001345,total_distance,0.000878,AIRLINE_airline_bdeeef3a675587d530de70a25d7118d2,0.000629,AIRCRAFT_NOx EI App (g/kg),0.000519,AIRLINE_airline_a73f82288988b79be490c6322f4c32ed,0.000426,vertical_rate_quant_4_stdv,0.000400,AIRLINE_airline_6351ec1b849adacc0cbb3b1313d8d39b,0.000289,AIRLINE_airline_2d5def0a5a844b343ba1b7cc

In [15]:
# predict and save
latest_version = max(
    [
        int(item.split("_v")[1].split("_")[0]) 
            for item in os.listdir(submissions_folder) 
                if "_v" in item
    ]
)
num=latest_version+1
team_name="team_diligent_igloo"
team_id="ad3e299d-a627-47d3-99a4-5cad9228e591"
file_name=f"{team_name}_v{num}_{team_id}.csv"
file_path = os.path.join(submissions_folder, file_name)

pred_df = xgb_X_y_df_final_sub[["flight_id", "tow"]].copy()

pred_df.tow = best_reg.predict(
    xgb_X_y_df_final_sub[xgb_X_y_df_final_sub.columns[1:-1]].values
)
pred_df.to_csv(os.path.join(submissions_folder, file_name), index=False)

In [17]:
# UPLOAD (WINDOWS POWERSHELL: my local way of doing it)
powershell_command_1 = f"export PATH=$PATH:$PWD/minio-binaries/"
powershell_command_2 = f'.\\mc alias set dc24 https://s3.opensky-network.org "ZG58zJvKhts2bkOX" "eU95azmBpK82kg96mE0TNzsWov3OvP2d"'
powershell_command_3 = f'.\\mc cp {file_path} dc24//submissions//{file_name}'

# for powershell_command in [powershell_command_1, powershell_command_2]:
#     print(powershell_command, "\n")
#     result = subprocess.run(["powershell", "-Command", powershell_command], capture_output=True, text=True)
#     result_output = result.stdout
#     result_error = result.stderr
#     #Output the result
#     print("Output:", result_output)
#     print("Error:", result_error)

#     print("\n\n")